# Testing area error

In [7]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
}


In [33]:
url_1 = 'https://www.rightmove.co.uk/properties/147576875#/?channel=RES_BUY'
url_2 = 'https://www.rightmove.co.uk/properties/147576131#/?channel=RES_BUY'
response = requests.get(url_2, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

In [34]:
property_card = soup.find_all('dd', class_='_3ZGPwl2N1mHAJH3cbltyWn')
info = [i.text.strip() for i in property_card]

if info[3] == 'Ask agent':
    info[3] = '0 sq ft 0 sq m'
else:
    info[3] = info[3].replace(' sq m', '').replace('sq ft', '').replace(',', '').split()

info[3]
info
# append the data to the lists
# sq_ft.append(info[3][0])
# sq_m.append(info[3][1])


['Flat', '2', '1', '0 sq ft 0 sq m', 'Leasehold']